In [1]:
%matplotlib nbagg

In [9]:
import numpy as np
import numba
import matplotlib.pyplot as plt

from libertem_live.detectors.k2is.proto import MsgReaderThread, make_tile
from libertem.io.dataset.k2is import DataBlock, SHUTTER_ACTIVE_MASK

In [3]:
packet_size = 0x5758

In [4]:
class MockThread:
    buffered_tile = None
    first_frame_id = 0
    timeout = 1
    # first sector
    x_offset = 0
    
    def __init__(self):
        self.packet_counter = 1
    
    def is_stopped(self):
        return False

In [5]:
x = 1
y_mult, xx = divmod(x, 16)
(240 - xx * 16, y_mult*930) 

(224, 0)

In [ ]:
class MockSocket:
    def __init__(self):
        self.block_count = 0    
        self.offsets = []
        # [240, 224, 208, ..., 16, 0]
        for x in range(240, -16, -16):
            self.offsets.append((x, 0))
        for x in range(240, -16, -16):
            self.offsets.append((x, 930))
    
    def recvmsg_into(self, buffers):
        frame, packet = divmod(self.block_count, 32)
        # The calling code always receives into a single buffer for a single packet
        assert len(buffers) == 1
        buffer = buffers[0]
        assert len(buffer) == packet_size
        
        x, y = self.offsets[packet]
        
        header = np.zeros(1, dtype=DataBlock.header_dtype)[0]
        header['sync'] = 0xFFFF0055
        header['version'] = 1
        header['flags'] = SHUTTER_ACTIVE_MASK
        header['block_count'] = self.block_count
        header['width'] = 256
        header['height'] = 1860
        header['frame_id'] = frame
        header['pixel_x_start'] = x
        header['pixel_y_start'] = y
        header['pixel_x_end'] = x + 15
        header['pixel_y_end'] = y + 929
        header['block_size'] = packet_size
        payload = bytearray(packet_size - header.nbytes)
        # Encode the block count as uint12 payload in the first 1 1/2 bytes
        tag = np.int16(self.block_count)
        b1 = tag & 0xFF
        b2 = (tag & 0x0F00) >> 8
        payload[0] = b1
        payload[1] = b2
        
        nbytes = packet_size
        ancdata = None
        msg_flags = None
        address = None
        buffer[:] = header.tobytes() + payload
        # Know where we are in the virtual dataset
        self.block_count += 1
        
        return (nbytes, ancdata, msg_flags, address)
    
    def settimeout(self, timeout):
        return
    

In [ ]:
thread = MockThread()
socket = MockSocket()

In [ ]:
packets = MsgReaderThread.read_loop_bulk(thread, socket)
tiles = MsgReaderThread.get_tiles(thread, packets, end_after_idx=8)

In [ ]:
FRAMES_PER_TILE = 4
for tile_idx, t in enumerate(tiles):
    for frame in range(FRAMES_PER_TILE):
        for y in (0, 930):
            for x in range(0, 256, 16):
                tag = t[frame, y, x]
                offset = 0 if y == 0 else 16
                # unwind the sequence from MockSocket
                target = 15 - x//16 + offset + 32*frame + 32*FRAMES_PER_TILE*tile_idx
                assert tag == target
                print(frame, y, x, tag)
                
        

In [ ]:
def test():
    def hello():
        return
    return hello()

In [ ]:
def hello():
    return

def test2():
    hello()

In [ ]:
%timeit test()

In [ ]:
%timeit test2()

In [ ]:
2**12 - 1

In [8]:
l = [
    make_tile((4, 1860, 256), 0),
    make_tile((4, 1860, 256), 4),
    make_tile((4, 1860, 256), 8)
]

In [21]:
l.append(make_tile((4, 1860, 256), 0))
%timeit numba.typed.List(l)

137 µs ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [20]:
%timeit numba.typed.List(l)

135 µs ± 2.78 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
